In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    WebDriverException,
)
import os
import time
import shutil
from alive_progress import alive_bar

In [2]:


def drop_file(filepath):
    try:
        # Vérifier si le fichier existe avant de le supprimer
        if os.path.exists(filepath):
            # Supprimer le fichier
            os.remove(filepath)
            print(f"Le fichier {filepath} a été supprimé avec succès.")
        else:
            print(f"Le fichier {filepath} n'existe pas.")
    except Exception as e:
        print(f"Une erreur s'est produite lors de la suppression du fichier : {e}")

# Effacer les anciens fichiers
drop_file(f"{os.getcwd()}/euromillions.csv")
drop_file("/Users/joeybruno/Downloads/euromillions.csv")

download_dir = f"/Users/joeybruno/Downloads"

# Créer la page web automatique sur Safari
driver = webdriver.Safari()
driver.get(
    "https://www.loterieplus.com/euromillions/services/telechargement-resultat.php"
)  # lien de la page web

try:
    # choix sur séparateur décimale
    select_dec = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "sd2"))
    )
    select_dec.click()

    # choix du séparateur de champ
    select_sep = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "sc2"))
    )
    select_sep.click()

    # simulation du click sur le bouton de téléchargement
    valid_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "valid"))
    )
    valid_button.click()

    time.sleep(2)  # délai de téléchargement

    # destination de l'enregistrement du fichier
    destination_file_path = os.path.join(
        f"{os.getcwd()}", "euromillions.csv"
    )

    if os.path.exists(f"{download_dir}/euromillions.csv"):
        # Copier le fichier dans le répertoire courant
        shutil.copy(f"{download_dir}/euromillions.csv", destination_file_path)
        print(f"Le fichier a été copié dans {destination_file_path}")
    else:
        print(f"Le fichier euromillions.csv n'existe pas.")

except TimeoutException:
    print("L'attente de l'élément a dépassé le temps imparti.")
except NoSuchElementException:
    print("L'élément spécifié est introuvable.")
except WebDriverException as e:
    print(f"Erreur WebDriver rencontrée: {e}")
except Exception as e:
    print(f"Une erreur inattendue s'est produite: {e}")

finally:
    driver.quit()

Le fichier /Users/joeybruno/Desktop/GITHUB/GITHUB - SELENIUM/euromillions.csv a été supprimé avec succès.
Le fichier /Users/joeybruno/Downloads/euromillions.csv a été supprimé avec succès.
Le fichier a été copié dans /Users/joeybruno/Desktop/GITHUB/GITHUB - SELENIUM/euromillions.csv


In [3]:
df = pd.read_csv("euromillions.csv", sep=";", 
                 usecols=["NUM","DATE","JACKPOT","N1","N2","N3","N4","N5","E1","E2","NBJEU"])
df

,NUM,DATE,JACKPOT,N1,N2,N3,N4,N5,E1,E2,NBJEU
0,1744,2024-06-07,130000000,15,16,26,30,37,5,8,42673215
1,1743,2024-06-04,70000000,6,7,9,14,43,3,4,19469756
2,1742,2024-05-31,61000000,4,7,16,33,34,7,8,24467994
3,1741,2024-05-28,48000000,16,18,35,36,41,6,7,17463825
4,1740,2024-05-24,38000000,9,12,18,22,50,1,3,22391485
...,...,...,...,...,...,...,...,...,...,...,...
1739,5,2004-03-12,15000000,15,24,28,44,47,4,5,12801928
1740,4,2004-03-05,20000000,4,7,33,37,39,1,5,14612108
1741,3,2004-02-27,17000000,14,18,19,31,37,4,5,13778241
1742,2,2004-02-20,15000000,7,13,39,47,50,2,5,14099001


In [4]:
df = df.dropna().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   NUM      1744 non-null   int64 
 1   DATE     1744 non-null   object
 2   JACKPOT  1744 non-null   int64 
 3   N1       1744 non-null   int64 
 4   N2       1744 non-null   int64 
 5   N3       1744 non-null   int64 
 6   N4       1744 non-null   int64 
 7   N5       1744 non-null   int64 
 8   E1       1744 non-null   int64 
 9   E2       1744 non-null   int64 
 10  NBJEU    1744 non-null   int64 
dtypes: int64(10), object(1)
memory usage: 150.0+ KB


In [5]:
numbers = pd.concat([df[col] for col in df.columns[3:-3]])
top_num = numbers.value_counts().sort_values(ascending=False).head()
top_num

23    201
44    197
21    196
19    196
50    193
Name: count, dtype: int64

In [6]:
etoiles = pd.concat([df[col] for col in df.columns[-3:-1]])
top_et = etoiles.value_counts().sort_values(ascending=False).head()
top_et

3    356
2    350
8    335
9    320
7    313
Name: count, dtype: int64

In [7]:
top_freq_num = [int(top_num.index[0]),int(top_num.index[1]),int(top_num.index[2]),int(top_num.index[3]),int(top_num.index[4])]
top_freq_et = [int(top_et.index[0]),int(top_et.index[1])]

tirage_freq = top_freq_num + top_freq_et
print(f"Tirage selon la fréquence de sortie: {tirage_freq}")

Tirage selon la fréquence de sortie: [23, 44, 21, 19, 50, 3, 2]


In [8]:
def check_pair(row, num1, num2):
    number_row = row[['N1', 'N2', 'N3', 'N4', 'N5']]
    return (num1 in number_row.values) and (num2 in number_row.values)

df_pair = pd.DataFrame(columns=["target","N1","N2","N3","N4","N5"])
n_rows = 0

with alive_bar(5,force_tty=True,title="Loading...") as bar:
    for best_num in tirage_freq[:5]:
        df_pair.loc[n_rows,"target"] = best_num
        results = {}
        
        for num in range(1, 51):
            count = df.apply(check_pair, axis=1, args=(best_num, num)).sum()
            results[num] = count

        # Trier les résultats par ordre croissant du nombre d'apparitions
        sorted_results = sorted(results.items(), key=lambda item: item[1], reverse=True)

        print('-----------')

        # Afficher les résultats triés
        n_col = 1
        for num, count in sorted_results[1:6]:
            df_pair.loc[n_rows,f"N{n_col}"] = num
            n_col += 1
            print(f'Le numéro {best_num} et le numéro {num} sont apparus ensemble {count} fois.')
        
        n_rows += 1
        bar()

on 0: -----------                                                                ▃▅▇ 0/5 [0%] in 0s (~0s, 0|                                        | ▅▃▁ 0/5 [0%] in 5s (~0s, 0 ▅▃▁ 0/5 [0%] in 19s (~0s,  ▆▄▂ 0/5 [0%] in 24s (~0s,  ▅▃▁ 0/5 [0%] in 24s (~0s, 
on 0: Le numéro 23 et le numéro 4 sont apparus ensemble 25 fois.                
on 0: Le numéro 23 et le numéro 32 sont apparus ensemble 25 fois.               
on 0: Le numéro 23 et le numéro 37 sont apparus ensemble 25 fois.               
on 0: Le numéro 23 et le numéro 44 sont apparus ensemble 23 fois.               
on 0: Le numéro 23 et le numéro 19 sont apparus ensemble 22 fois.               
on 1: -----------                                                               
on 1: Le numéro 44 et le numéro 12 sont apparus ensemble 24 fois.               
on 1: Le numéro 44 et le numéro 15 sont apparus ensemble 24 fois.               
on 1: Le numéro 44 et le numéro 39 sont apparus ensemble 24 fois.               
on 1: Le numé

In [9]:
df_pair

,target,N1,N2,N3,N4,N5
0,23,4,32,37,44,19
1,44,12,15,39,23,35
2,21,14,45,26,17,19
3,19,12,10,23,37,21
4,50,3,26,44,4,10


In [10]:
numbers_df2 = pd.concat([df_pair[col] for col in df_pair.columns[1:]])
nb_count = numbers_df2.value_counts()
most_occ = nb_count[nb_count == 2].index.tolist()
most_occ

[4, 10, 19, 44, 23, 37, 12, 26]

In [11]:
communs = set(tirage_freq[:-2]) & set(most_occ)
print("Éléments communs :", communs)

Éléments communs : {19, 44, 23}
